# Multi-Agent Order Management System with MongoDB

In this example, we will implement a multi-agent system for managing product orders, inventory, and deliveries using
- `smoalagents` for agent management
- MongoDB for data persistence
- DeepSeek Chat as the LLM model

## Setups

In [ ]:
!pip install smolagents pymongo litellm

In [ ]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, ManagedAgent, CodeAgent

from pymongo import MongoClient
from datatime import datetime
from typing import List, Dict, Optional

from google.colab import userdata

MODEL_ID = 'deepseek/deepseek-chat'
MONGODB_URI = userdata.get('MONGO_URI')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

## Database connection class

We will create a MongoDB connection manager:

In [ ]:
mongoclient = MongoClient(
    MONGODB_URI,
    appname='devrel.showcase.multi-smolagents'
)
db = mongoclient.warehouse

## Agent Tool Definitions

We will implement three tools for warehouse management:

```
Inventory Management Tools:
+-------------------+-------------------+
| Tool              | Description       |
+-------------------+-------------------+
| check_stock       | Queries stock     |
|                   | levels            |
+-------------------+-------------------+
| update_stock      | Adjusts inventory |
|                   | quantities        |
+-------------------+-------------------+

Order Management Tools:
+-------------------+-------------------+
| Tool              | Description       |
+-------------------+-------------------+
| create_order      | Creates new order |
|                   | document          |
+-------------------+-------------------+

Delivery Management Tools:
+-------------------+-------------------+
| Tool              | Description       |
+-------------------+-------------------+
| update_delivery   | Updates delivery  |
| _status           | status            |
+-------------------+-------------------+

Decision Flow:
+-------------------+-------------------+
| Step              | Action            |
+-------------------+-------------------+
| 1. Create Order   | Uses `create_order`|
|                   | tool to create    |
|                   | order document    |
+-------------------+-------------------+
| 2. Update Stock   | Uses `update_stock`|
|                   | tool to adjust    |
|                   | inventory         |
+-------------------+-------------------+
| 3. Update Delivery| Uses `update_delivery`|
| Status            | _status tool to   |
|                   | set delivery      |
|                   | status to         |
|                   | `in_transit`      |
+-------------------+-------------------+
```

In [ ]:
# ===== Inventory Management Tools =====

@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})


@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one(
        {"_id": product_id},
        {"$inc": {"quantity": -quantity}}
    )
    return result.modified_count > 0

In [ ]:
# ===== Order Management Tool =====

@tool
def create_order(products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message
    """
    order = {
        'products': products,
        'status': 'pending',
        'delivery_address': address,
        'created_at': datetime.now()
    }
    result = db.orders.insert_one(order)
    return f"Successfully ordered: {str(result.inserted_id)}"

In [ ]:
# ===== Delivery Management Tool =====

from bson.objectid import ObjectId

@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided.

    Args:
        order_id: Order identifier
        status: New delivery status is set to in_transit or delivered

    Returns:
        bool: Success status
    """
    if status not in ['pending', 'in_transit', 'delivered', 'cancelled']:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one(
        {"_id": ObjectId(order_id), 'status': 'pending'},
        {"$set": {'status': status}}
    )
    return result.modified_count > 0

## Main order management system

We will implement a multi-agent architecture for order processing with the following components:
- Inventory Agent - handles stock checking and updates
- Order Agent - manages order creation and documentation
- Delivery Agent - controls order delivery status changes
- Manager Agent - orchestrates workflow between other agents

The system contains the following process flow:
1. Create order documents for customer requests
2. Verify and update product inventory levels
3. Initialize delivery tracking status
4. Coordinate agent interactions through the manager

In [ ]:
class OrderManagementSystem:
    """Multi-agent order management system."""

    def __init__(self, model_id: str = MODEL_ID):
        self.model = LiteLLMModel(
            model_id=model_id,
            api_key=DEEPSEEK_API_KEY
        )

        # Create agents
        self.inventory_agent = ToolCallingAgent(
            tools=[check_stock, update_stock],
            model=self.model,
            max_iterations=10
        )
        self.order_agent = ToolCallingAgent(
            tools=[create_order],
            model=self.model,
            max_iterations=10
        )
        self.delivery_agent = ToolCallingAgent(
            tools=[update_delivery_status],
            model=self.model,
            max_iterations=10
        )

        # Manage created agents
        self.managed_agents = [
            ManagedAgent(self.inventory_agent, 'inventory', 'Manages product inventory'),
            ManagedAgent(self.order_agent, 'order', 'Handles order creation'),
            ManagedAgent(self.delivery_agent, 'delivery', 'Manages delivery status')
        ]

        # Create manager agent
        self.manager = CodeAgent(
            tools=[],
            system_prompt="""For each order:
            1. Create the order document
            2. Update the inventory
            3. Set deliviery status to in_transit

            Use relevant agents:  {{managed_agents_descriptions}}  and you can use {{authorized_imports}}
            """,
            model=self.model,
            managed_agents=self.managed_agents,
            additional_authorized_imports=['time', 'json']
        )

    def process_order(self, orders: List[Dict]) -> str:
        """Process a set of orders.

        Args:
            orders: List of orders each has address and products

        Returns:
            str: Processing result
        """
        return self.manager.run(
            f"""Process the following  {orders} as well as substract the ordered items from inventory.
            to be delivered to relevant addresses
            """
        )

## Add sample data

To test our order management system, we can populate the MongoDB database with sample product data. The sample data provides a representative mix of electronics products with varying price points and stock levels to demonstrate inventory tracking.

In [ ]:
def add_sample_products():
    db.products.delete_many({})

    sample_products = [
        {"_id": "prod1", "name": "Laptop", "price": 999.99, "quantity": 10},
        {"_id": "prod2", "name": "Smartphone", "price": 599.99, "quantity": 15},
        {"_id": "prod3", "name": "Headphones", "price": 99.99, "quantity": 30},
    ]

    db.products.insert_many(sample_products)
    print("Sample products added successfully!")


add_sample_products()

## Test the system

In [ ]:
# Initialize system
system = OrderManagementSystem()

# Create test orders
test_orders = [
    {
        "products": [{"product_id": "prod1", "quantity": 2}, {"product_id": "prod2", "quantity": 1}],
        "address": "123 Main St",
    },
    {"products": [{"product_id": "prod3", "quantity": 3}], "address": "456 Elm St"},
]

# Process order
result = system.process_order(test_orders)

print("Orders processing result:", result)